<p style="text-align: center">
    <img src="../../assets/images/untref-logo-negro.svg" style="height: 50px;" />
</p>

<h3 style="text-align: center">Estructuras de Datos</h3>

<h2 style="text-align: center">Clase 6: Camino mínimo en grafos dirigidos</h3>

## Camino Mínimo

Dado un grafo dirigido encontrar el camino mínimo desde un vértice dado a todos los demás vértices del grafo.

### Métodos

* Grafo dirigido sin pesos: Se resuelve con un recorrido **BFS**.
* Grafo dirigido con pesos no negativos: **Algoritmo de Dijkstra**.
* Grafo dirigido con pesos negativos: **Algoritmo de Bellman-Ford**


### Algoritmo de Dijkstra

Encontrar el camino mínimo desde $A$ hacia todos los demás vértices del grafo:

<p style="text-align: center">
    <img src="figuras/grafo-dijkstra.png" style="width: 500px;" />
</p>

|Referencia|Descripción|
|--:|---|
|`s`| Nodo inicial|
|`distancia[v]`|Distancia desde `s` hasta el nodo `v`|
|`previo[v]`|Empezando por `s`, guarda cual es el nodo anterior a `v` en el camino `s-v`|
|`visitado[v]`|Verdadero si `v` ya fué visitado|

#### Ejemplo

In [ ]:
from edd.grafo import DiGrafo

G = DiGrafo()
G.agregar_arista("A", "B", 4)
G.agregar_arista("A", "C", 1)
G.agregar_arista("B", "E", 3)
G.agregar_arista("C", "B", 2)
G.agregar_arista("C", "D", 2)
G.agregar_arista("D", "E", 3)

G.draw()

In [ ]:
from math import inf

from edd.coladeprioridad import ColaDePrioridad
from edd.grafo import Vertice


def dijkstra_por_pasos(self, s: Vertice):
    pq = ColaDePrioridad()

    distancia = {v: inf for v in self.vertices}
    previo = {v: None for v in self.vertices}
    visitado = {v: False for v in self.vertices}
    aristas = []

    distancia[s] = 0
    pq.encolar(s, distancia[s])

    ### <IGNORAR>
    yield {
        "msj": f"Encolamos {s.id} como nodo inicial",
        "cola": pq,
        "distancia": distancia,
        "previo": previo,
        "visitado": visitado,
        "aristas_recorridas": aristas,
        "v": None,
        "w": None,
    }
    ### </IGNORAR>

    while not pq.esta_vacia():
        v, _ = pq.desencolar()

        visitado[v] = True

        ### <IGNORAR>
        yield {
            "msj": f"Desencolamos y visitamos {v.id}",
            "cola": pq,
            "distancia": distancia,
            "previo": previo,
            "visitado": visitado,
            "aristas_recorridas": aristas,
            "v": v,
            "w": None,
        }
        ### </IGNORAR>

        for a in v.aristas:
            aristas.append((a.origen.id, a.destino.id))
            w = a.destino

            if not visitado[w] and distancia[v] + a.peso < distancia[w]:
                distancia[w] = distancia[v] + a.peso
                previo[w] = v
                pq.encolar(w, distancia[w])
                msj = f"Encolamos {w.id} con la nueva distancia {distancia[w]}"
            else:
                msj = f"No se mejoró la distancia a {w.id}"

            ### <IGNORAR>
            yield {
                "msj": msj,
                "cola": pq,
                "distancia": distancia,
                "previo": previo,
                "visitado": visitado,
                "aristas_recorridas": aristas,
                "v": v,
                "w": w,
            }
            ### </IGNORAR>


# Hacemos "monkey patching" del método que acabamos de implementar.
DiGrafo.dijkstra_por_pasos = dijkstra_por_pasos

In [ ]:
# Ejecutar esto una vez para inicializar el Orden Topológico paso a paso.
pasos = G.dijkstra_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo de Dijkstra.
from edd.jp import build_html_table, render_html

try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n\n")
finally:
    headers = ["Vertice", "Distancia", "Previo", "Visitado"]
    rows = [
        [
            v.id,
            estado["distancia"][v],
            estado["previo"][v].id if estado["previo"][v] else "-",
            "Si" if estado["visitado"][v] else "No",
        ]
        for v in G.vertices
    ]
    print(f">>> {estado['msj']}\n")
    print(f"pq = {estado['cola']}\n")
    print(f"v = {estado['v'].id if estado['v'] else ''}")
    print(f"w = {estado['w'].id if estado['w'] else ''}")
    render_html(build_html_table(headers, rows))

    G.draw(
        highlight_edges=estado["aristas_recorridas"],
        highlight_nodes=[v.id for v, visitado in estado["visitado"].items() if visitado],
    )